
# OpSpam Replication: RoBERTa(+LSTM) **and** TF‑IDF + Logistic Baseline

This notebook is pre-configured for your CSV with columns:
- **`text`**: review text
- **`deceptive`**: binary label (1 = deceptive, 0 = truthful)

It trains **two models** and reports comparable metrics/plots:
1. **Baseline:** TF‑IDF + Logistic Regression (mirrors your demo)
2. **RoBERTa (+ optional LSTM)** classifier

Exports: metrics CSVs, confusion matrix, ROC and PR curves, and a side‑by‑side table.


# Setup

In [ ]:

#@title ⬇️ Install dependencies
!pip -q install transformers==4.44.2 datasets==2.21.0 accelerate==0.34.2 scikit-learn==1.5.1 matplotlib==3.9.0 torch==2.3.1 -U

import os, random, math, json, sys, time, re
from dataclasses import dataclass
from typing import Optional, Dict, Any

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, average_precision_score,
    classification_report, roc_curve, precision_recall_curve
)

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from transformers import (
    AutoTokenizer, AutoModel, AutoConfig,
    TrainingArguments, Trainer
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)


# Configuration

In [ ]:

#@title ⚙️ Configuration
MODEL_NAME = "roberta-base"  #@param ["roberta-base"]
MAX_LEN = 256  #@param {type:"integer"}
BATCH_SIZE = 32  #@param {type:"integer"}
EPOCHS = 8  #@param {type:"integer"}
LEARNING_RATE = 2e-5  #@param {type:"number"}
WEIGHT_DECAY = 0.01  #@param {type:"number"}
SEED = 42  #@param {type:"integer"}
USE_LSTM = True  #@param {type:"boolean"}
LSTM_HIDDEN = 128  #@param {type:"integer"}
DROPOUT = 0.6  #@param {type:"number"}

# Baseline config
TFIDF_MAX_FEATURES = 5000  #@param {type:"integer"}

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)


# Data

In [ ]:

#@title 📥 Load data
csv_path = ""  #@param {type:"string"}

if not csv_path or not os.path.exists(csv_path):
    try:
        from google.colab import files  # type: ignore
        print("Upload 'deceptive-opinion.csv'")
        uploaded = files.upload()
        csv_path = list(uploaded.keys())[0]
    except Exception as e:
        print("No Colab file picker. Falling back to default name.")
        csv_path = "deceptive-opinion.csv"

assert os.path.exists(csv_path), f"CSV not found at {csv_path}"
df = pd.read_csv(csv_path)
print("Columns:", df.columns.tolist())
display(df.head(3))


In [ ]:
LABEL_COL = "deceptive"  # make sure this is the right column name

print("dtype:", df[LABEL_COL].dtype)
print("\nRaw unique values (repr):")
raw_uniques = sorted({repr(x) for x in df[LABEL_COL].unique()})
for u in raw_uniques:
    print("  ", u)

print("\nValue counts (dropna=False):")
print(df[LABEL_COL].value_counts(dropna=False))


# Columns & Quick Clean

In [ ]:
import unicodedata
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

TEXT_COL  = "text"
LABEL_COL = "deceptive"   # name is okay; values are strings

# 1) Clean text like your demo (does NOT touch labels)
df[TEXT_COL] = (
    df[TEXT_COL].astype(str)
                .str.lower()
                .str.replace(r"[^\w\s]", "", regex=True)
)

# 2) Robust label normalization → map to {0,1}
def norm(s):
    s = unicodedata.normalize("NFKC", str(s))
    return s.replace("\u00A0"," ").strip().lower()   # strip NBSP and whitespace

lab_norm = df[LABEL_COL].apply(norm)

mapping = {"truthful": 0, "deceptive": 1, "0": 0, "1": 1}
df[LABEL_COL] = lab_norm.map(mapping)

# 3) Fail early if anything didn’t map
if df[LABEL_COL].isna().any():
    bad = sorted(df.loc[df[LABEL_COL].isna(), LABEL_COL].unique())
    raise ValueError(f"Unrecognized label strings: {bad} — extend the mapping.")

# 4) Verify full dataset has both classes
print("Full dataset counts:", df[LABEL_COL].value_counts().sort_index().to_dict())
assert set(df[LABEL_COL].unique()) == {0,1}, "Dataset collapsed to one class."

# 5) Stratified split (guarantees both classes appear in each split for balanced data)
X = df[TEXT_COL].values
y = df[LABEL_COL].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.10, random_state=42, stratify=y_train
)

def check_split(y, name):
    uniq = np.unique(y)
    print(f"{name} classes:", uniq, "counts:", {int(c): int((y==c).sum()) for c in uniq})
    assert set(uniq) == {0,1}, f"{name} has only one class ({uniq})."

check_split(y,       "All")
check_split(y_train, "Train")
check_split(y_val,   "Val")
check_split(y_test,  "Test")


# Split

In [ ]:

#@title 🔀 Stratified split (80/20) and make a val split from train
X = df[TEXT_COL].values
y = df[LABEL_COL].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=SEED, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.10, random_state=SEED, stratify=y_train
)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")


In [ ]:
def check_split(y, name):
    uniq = np.unique(y)
    print(f"{name} classes:", uniq)
    assert set(uniq) == {0,1}, (
        f"{name} has only one class ({uniq}). "
        "Check label mapping/stratification or class distribution."
    )

check_split(y_train, "Train")
check_split(y_val,   "Val")
check_split(y_test,  "Test")


# Baseline (TF‑IDF + Logistic)

# Tokenizer & Datasets

In [ ]:

#@title 🔡 Tokenize for RoBERTa
from transformers import AutoTokenizer, AutoModel, AutoConfig
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts; self.labels = labels
        self.tokenizer = tokenizer; self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]), padding="max_length", truncation=True,
            max_length=self.max_len, return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(int(self.labels[idx]), dtype=torch.long)
        return item

train_ds = TextDataset(X_train, y_train, tokenizer, MAX_LEN)
val_ds   = TextDataset(X_val, y_val, tokenizer, MAX_LEN)
test_ds  = TextDataset(X_test, y_test, tokenizer, MAX_LEN)


# Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaModel

class RobertaLSTMAttentionClassifier(nn.Module):  #Novel ml model with attention
    def __init__(self,
                 model_name: str = "roberta-base",
                 lstm_hidden: int = 128,
                 dropout: float = 0.6,
                 use_lstm: bool = True,   # keep for compatibility
                 num_labels: int = 2):
        super().__init__()

        self.num_labels = num_labels
        self.use_lstm = use_lstm

        # RoBERTa encoder (same as before)
        self.roberta = RobertaModel.from_pretrained(model_name)
        hidden_size = self.roberta.config.hidden_size

        if use_lstm:
            self.lstm = nn.LSTM(
                input_size=hidden_size,
                hidden_size=lstm_hidden,
                num_layers=1,
                batch_first=True,
                bidirectional=True
            )
            feat_dim = lstm_hidden * 2  # biLSTM
        else:
            # fall back to using RoBERTa hidden size directly
            self.lstm = None
            feat_dim = hidden_size

        # Self-attention over sequence features
        self.attention = nn.Linear(feat_dim, 1, bias=False)

        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(feat_dim, num_labels)

    def forward(self,
                input_ids=None,
                attention_mask=None,
                labels=None,
                **kwargs):

        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        seq = outputs.last_hidden_state        # [B, T, H]

        if self.use_lstm:
            lstm_out, _ = self.lstm(seq)       # [B, T, 2*hidden]
            feats = lstm_out
        else:
            feats = seq                        # [B, T, H]

        # self-attention pooling
        attn_scores = self.attention(feats).squeeze(-1)  # [B, T]

        if attention_mask is not None:
            attn_scores = attn_scores.masked_fill(attention_mask == 0, -1e9)

        attn_weights = F.softmax(attn_scores, dim=-1)    # [B, T]
        attn_weights = attn_weights.unsqueeze(1)         # [B, 1, T]
        context = torch.bmm(attn_weights, feats).squeeze(1)  # [B, feat_dim]

        context = self.dropout(context)
        logits = self.classifier(context)                # [B, num_labels]

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels),
                            labels.view(-1))

        if loss is not None:
            return {"loss": loss, "logits": logits}
        else:
            return {"logits": logits}


In [ ]:

#@title 🧠 RoBERTa(+LSTM) model
class RobertaLSTMClassifier(nn.Module):
    def __init__(self, model_name, lstm_hidden=128, dropout=0.6, use_lstm=True):
        super().__init__()
        self.use_lstm = use_lstm
        self.config = AutoConfig.from_pretrained(model_name)
        self.roberta = AutoModel.from_pretrained(model_name, add_pooling_layer=False)
        hidden = self.config.hidden_size
        if self.use_lstm:
            self.lstm = nn.LSTM(input_size=hidden, hidden_size=lstm_hidden,
                                num_layers=1, batch_first=True, bidirectional=False)
            feat = lstm_hidden
        else:
            feat = hidden
        self.bn = nn.BatchNorm1d(feat)
        self.drop = nn.Dropout(dropout)
        self.cls = nn.Linear(feat, 2)

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        out = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        last = out.last_hidden_state
        if self.use_lstm:
            _, (h, _) = self.lstm(last)
            feat = h.squeeze(0)
        else:
            feat = last[:,0,:]
        feat = self.bn(feat)
        feat = self.drop(feat)
        logits = self.cls(feat)
        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
        return {"loss": loss, "logits": logits}


In [ ]:
# from transformers import TrainingArguments, Trainer

# model = RobertaLSTMClassifier(MODEL_NAME, LSTM_HIDDEN, DROPOUT, USE_LSTM)

# # Count parameters (total & trainable) — works for any PyTorch model
# import pandas as pd
# import torch.nn as nn

# def count_params(m: nn.Module):
#     trainable = sum(p.numel() for p in m.parameters() if p.requires_grad)
#     total     = sum(p.numel() for p in m.parameters())
#     return trainable, total

# rows = []
# # Whole model
# tr, tt = count_params(model)
# rows.append({"Module": "TOTAL", "Trainable": tr, "Total": tt})

# # If you want the split (adjust names to your class if different)
# enc = getattr(model, "roberta", None) or getattr(model, "encoder", None)
# lstm = getattr(model, "lstm", None)
# cls  = getattr(model, "classifier", None)
# bn   = getattr(model, "bn", None)
# drop = getattr(model, "dropout", None)

# for name, part in [("Encoder (RoBERTa)", enc), ("Head::lstm", lstm),
#                    ("Head::bn", bn), ("Head::dropout", drop), ("Head::classifier", cls)]:
#     if part is not None:
#         tr, tt = count_params(part)
#         rows.append({"Module": name, "Trainable": tr, "Total": tt})

# df_params = pd.DataFrame(rows)
# df_params["Trainable (M)"] = (df_params["Trainable"]/1e6).round(3)
# df_params["Total (M)"]     = (df_params["Total"]/1e6).round(3)
# print(df_params)
# df_params.to_csv("parameter_counts.csv", index=False)
# print("Saved parameter_counts.csv")


In [ ]:
# # --- C. Network schematic to PNG ---
# # If not installed:
# %pip install torchviz graphviz

# import torch
# from torchviz import make_dot

# # build one dummy batch
# dummy = tokenizer(["diagram probe"]*2, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
# dummy = {k: v.to(device) for k,v in dummy.items()}

# model.eval()
# out = model(**dummy)
# logits = out["logits"] if isinstance(out, dict) else out.logits

# # Create graph (params=True to show parameter nodes)
# dot = make_dot(logits, params=dict(model.named_parameters()))
# dot.format = "png"
# png_path = dot.render("roberta_lstm_schematic")  # produces roberta_lstm_schematic.png
# print("Saved:", png_path)


# Train (RoBERTa)

In [ ]:
#@title 🚀 Train RoBERTa(+LSTM)
from transformers import TrainingArguments, Trainer

model = RobertaLSTMClassifier(MODEL_NAME, LSTM_HIDDEN, DROPOUT, USE_LSTM)

def compute_metrics_fn(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    probs = torch.softmax(torch.tensor(preds), dim=-1).numpy()[:,1]
    y_pred = (probs >= 0.5).astype(int)
    y_true = p.label_ids

    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }

    # Check if there is more than one class in y_true before calculating AUROC and AP
    if len(np.unique(y_true)) > 1:
        metrics["auroc"] = roc_auc_score(y_true, probs)
        metrics["ap"] = average_precision_score(y_true, probs)
    else:
        print("Warning: Only one class present in y_true. Skipping AUROC and AP calculation.")
        metrics["auroc"] = float('nan') # Use NaN to indicate missing value
        metrics["ap"] = float('nan') # Use NaN to indicate missing value


    return metrics

args = TrainingArguments(
    output_dir="outputs_roberta",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    logging_steps=50,
    report_to=[], seed=SEED
)

trainer = Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=val_ds, compute_metrics=compute_metrics_fn)
trainer.train()

In [ ]:
###new trainer model
#@title 🚀 Train RoBERTa(+LSTM)
from transformers import TrainingArguments, Trainer

model = RobertaLSTMAttentionClassifier(
    MODEL_NAME,
    LSTM_HIDDEN,
    DROPOUT,
    USE_LSTM
)

def compute_metrics_fn(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    probs = torch.softmax(torch.tensor(preds), dim=-1).numpy()[:,1]
    y_pred = (probs >= 0.5).astype(int)
    y_true = p.label_ids

    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }

    # Check if there is more than one class in y_true before calculating AUROC and AP
    if len(np.unique(y_true)) > 1:
        metrics["auroc"] = roc_auc_score(y_true, probs)
        metrics["ap"] = average_precision_score(y_true, probs)
    else:
        print("Warning: Only one class present in y_true. Skipping AUROC and AP calculation.")
        metrics["auroc"] = float('nan') # Use NaN to indicate missing value
        metrics["ap"] = float('nan') # Use NaN to indicate missing value


    return metrics

args = TrainingArguments(
    output_dir="outputs_roberta",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    logging_steps=50,
    report_to=[], seed=SEED
)

trainer = Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=val_ds, compute_metrics=compute_metrics_fn)
trainer.train()

In [ ]:
# === Evaluate RoBERTa(+LSTM) on test set and save everything ===
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
    roc_auc_score, average_precision_score,
    roc_curve, precision_recall_curve
)
import pandas as pd

# 1) Predict probabilities and hard labels on test set
pred = trainer.predict(test_ds)
logits = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()[:, 1]
y_true = pred.label_ids
y_pred = (probs >= 0.5).astype(int)

# 2) Core metrics
acc  = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec  = recall_score(y_true, y_pred, zero_division=0)
f1   = f1_score(y_true, y_pred, zero_division=0)

print("=== RoBERTa(+LSTM) — Test Metrics (τ=0.5) ===")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1       : {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_true, y_pred, digits=4))

# 3) AUROC & AP (guard for single-class y_true)
has_both_classes = len(np.unique(y_true)) > 1
if has_both_classes:
    auroc = roc_auc_score(y_true, probs)
    ap    = average_precision_score(y_true, probs)
    print(f"AUROC    : {auroc:.4f}")
    print(f"AP       : {ap:.4f}")
else:
    auroc, ap = float('nan'), float('nan')
    print("Warning: Only one class present in y_true — skipping AUROC and AP.")

# 4) Confusion matrix (counts) + save figure
cm = confusion_matrix(y_true, y_pred)
plt.figure()
plt.imshow(cm, interpolation='nearest')
plt.title("Confusion Matrix — RoBERTa(+LSTM)")
plt.colorbar()
ticks = np.arange(2)
plt.xticks(ticks, ["Truthful (0)","Deceptive (1)"])
plt.yticks(ticks, ["Truthful (0)","Deceptive (1)"])
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center")
plt.ylabel("True label"); plt.xlabel("Predicted label")
plt.tight_layout()
plt.savefig("cm_roberta_lstm.png", dpi=160)
plt.show()

# 5) ROC curve (if valid) + save
if has_both_classes:
    fpr, tpr, _ = roc_curve(y_true, probs)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUROC = {auroc:.3f}")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title("ROC Curve — RoBERTa(+LSTM)")
    plt.legend(loc="lower right")
    plt.grid(True, linestyle=":")
    plt.tight_layout()
    plt.savefig("roc_curve_roberta_lstm.png", dpi=160)
    plt.show()

# 6) Precision–Recall curve (if valid) + save
if has_both_classes:
    precisions, recalls, _ = precision_recall_curve(y_true, probs)
    plt.figure()
    plt.plot(recalls, precisions, label=f"AP = {ap:.3f}")
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title("Precision–Recall Curve — RoBERTa(+LSTM)")
    plt.legend(loc="lower left")
    plt.grid(True, linestyle=":")
    plt.tight_layout()
    plt.savefig("pr_curve_roberta_lstm.png", dpi=160)
    plt.show()

# 7) Save metrics to CSV for reporting
metrics_row = {
    "accuracy": acc, "precision": prec, "recall": rec, "f1": f1,
    "auroc": auroc, "ap": ap, "threshold": 0.5,
    "seed": SEED, "use_lstm": USE_LSTM,
    "max_len": MAX_LEN, "epochs": EPOCHS,
    "batch_size": BATCH_SIZE, "learning_rate": LEARNING_RATE, "weight_decay": WEIGHT_DECAY,
}
pd.DataFrame([metrics_row]).to_csv("roberta_lstm_test_metrics.csv", index=False)
print("Saved: roberta_lstm_test_metrics.csv, cm_roberta_lstm.png",
      "(and ROC/PR curves if both classes present).")

# 8) Add to your comparison dict like other models
results = globals().get("results", {})
results["RoBERTa_LSTM"] = (acc, f1)
print("\nStored in results['RoBERTa_LSTM'] for comparison table.")


In [ ]:
# # 📚 Classical baselines on TF-IDF features
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC
# from sklearn.linear_model import LogisticRegression, SGDClassifier
# from sklearn.metrics import accuracy_score, f1_score

# def run_baseline(model_name, model, ngram_range=(1,1), use_liwc=False):
#     print(f"\n=== {model_name} ===")
#     # TF-IDF setup
#     tfidf = TfidfVectorizer(stop_words='english', max_features=5000, ngram_range=ngram_range)
#     X_train_t = tfidf.fit_transform(X_train)
#     X_test_t  = tfidf.transform(X_test)

#     # (Optional) LIWC features placeholder
#     # if use_liwc:  add psycholinguistic features here

#     model.fit(X_train_t, y_train)
#     y_pred = model.predict(X_test_t)
#     acc = accuracy_score(y_test, y_pred)
#     f1  = f1_score(y_test, y_pred)
#     print(f"Accuracy={acc:.4f}  F1={f1:.4f}")
#     return acc, f1

# #results = {}

# # SVM (unigram)
# results["SVM_unigram"] = run_baseline(
#     "SVM (unigram)", LinearSVC(), ngram_range=(1,1))

# # SVM (bigram)
# results["SVM_bigram"] = run_baseline(
#     "SVM (bigram)", LinearSVC(), ngram_range=(2,2))

# # SAGA (logistic regression solver)
# results["SAGA"] = run_baseline(
#     "Logistic (SAGA)", LogisticRegression(max_iter=2000, solver="saga"))

# # You can approximate 'LIWC' versions later if you have LIWC lexicon counts.


In [ ]:
# #@title ⬇️ Install and imports
# !pip -q install transformers==4.44.2 scikit-learn==1.5.1 matplotlib==3.9.0 torch==2.3.1 -U

# import os, re, unicodedata, random, numpy as np, pandas as pd, torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.decomposition import TruncatedSVD
# from sklearn.decomposition import LatentDirichletAllocation as LDA
# from transformers import AutoTokenizer

# device = "cuda" if torch.cuda.is_available() else "cpu"
# print("Device:", device)

# SEED = 42
# random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

In [ ]:

# #@title 🔡 Tokenization & Dataloaders
# MAX_LEN = 128  #@param {type:"integer"}
# tok = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

# def encode_texts(texts, max_len=MAX_LEN):
#     enc = tok(pd.Series(texts).tolist(), truncation=True, padding=True, max_length=max_len, return_tensors="pt")
#     return enc["input_ids"], enc["attention_mask"]

# train_ids, train_mask = encode_texts(X_train)
# val_ids,   val_mask   = encode_texts(X_val)
# test_ids,  test_mask  = encode_texts(X_test)

# ytr = torch.tensor(y_train, dtype=torch.long)
# yva = torch.tensor(y_val, dtype=torch.long)
# yte = torch.tensor(y_test, dtype=torch.long)

# train_loader = DataLoader(TensorDataset(train_ids, train_mask, ytr), batch_size=32, shuffle=True)
# val_loader   = DataLoader(TensorDataset(val_ids, val_mask, yva), batch_size=32)
# test_loader  = DataLoader(TensorDataset(test_ids, test_mask, yte), batch_size=32)

# VOCAB_SIZE = tok.vocab_size
# EMBED_DIM  = 128
# NUM_CLASSES = 2


In [ ]:

# #@title 🧪 Training helpers
# import torch.nn.functional as F

# def train_epoch(model, loader, optim, criterion):
#     model.train()
#     total = 0.0
#     for ids, mask, labels in loader:
#         ids = ids.to(device); labels = labels.to(device)
#         optim.zero_grad()
#         logits = model(ids)
#         loss = criterion(logits, labels)
#         loss.backward(); optim.step()
#         total += loss.item()*ids.size(0)
#     return total/len(loader.dataset)

# @torch.no_grad()
# def eval_model(model, loader):
#     model.eval()
#     preds=[]; trues=[]
#     for ids, mask, labels in loader:
#         ids=ids.to(device)
#         p = F.softmax(model(ids), dim=-1)[:,1].cpu().numpy()
#         yhat = (p>=0.5).astype(int)
#         preds.extend(list(yhat)); trues.extend(list(labels.numpy()))
#     return accuracy_score(trues, preds), f1_score(trues, preds)

# def fit_model(model, train_loader, val_loader, epochs=5, lr=1e-3, wd=1e-4):
#     model = model.to(device)
#     opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
#     crit = nn.CrossEntropyLoss()
#     for ep in range(1, epochs+1):
#         tr_loss = train_epoch(model, train_loader, opt, crit)
#         va_acc, va_f1 = eval_model(model, val_loader)
#         print(f"Epoch {ep}: train_loss={tr_loss:.4f}  val_acc={va_acc:.4f}  val_f1={va_f1:.4f}")
#     te_acc, te_f1 = eval_model(model, test_loader)
#     return te_acc, te_f1


In [ ]:

# #@title 🧠 TextCNN
# class TextCNN(nn.Module):
#     def __init__(self, vocab_size, embed_dim=128, num_classes=2, kernels=(3,4,5), channels=128, dropout=0.5):
#         super().__init__()
#         self.emb = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
#         self.convs = nn.ModuleList([nn.Conv1d(embed_dim, channels, k) for k in kernels])
#         self.drop = nn.Dropout(dropout)
#         self.fc = nn.Linear(channels*len(kernels), num_classes)
#     def forward(self, ids):
#         x = self.emb(ids).transpose(1,2)
#         outs = [F.relu(c(x)).max(dim=2)[0] for c in self.convs]
#         x = torch.cat(outs, dim=1)
#         x = self.drop(x)
#         return self.fc(x)

# textcnn = TextCNN(VOCAB_SIZE, EMBED_DIM, NUM_CLASSES)
# acc_textcnn, f1_textcnn = fit_model(textcnn, train_loader, val_loader, epochs=5)
# print(f"TextCNN Test: Acc={acc_textcnn:.4f}  F1={f1_textcnn:.4f}")


In [ ]:

# #@title 🧠 RCNN (BiLSTM + Conv)
# class RCNN(nn.Module):
#     def __init__(self, vocab_size, embed_dim=128, lstm_hidden=128, conv_channels=128, kernels=(3,4), num_classes=2, dropout=0.5):
#         super().__init__()
#         self.emb = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
#         self.lstm = nn.LSTM(embed_dim, lstm_hidden, batch_first=True, bidirectional=True)
#         self.convs = nn.ModuleList([nn.Conv1d(2*lstm_hidden, conv_channels, k) for k in kernels])
#         self.drop = nn.Dropout(dropout)
#         self.fc = nn.Linear(conv_channels*len(kernels), num_classes)
#     def forward(self, ids):
#         x = self.emb(ids)
#         x,_ = self.lstm(x)
#         x = x.transpose(1,2)
#         outs = [F.relu(c(x)).max(dim=2)[0] for c in self.convs]
#         x = torch.cat(outs, dim=1)
#         x = self.drop(x)
#         return self.fc(x)

# rcnn = RCNN(VOCAB_SIZE, EMBED_DIM)
# acc_rcnn, f1_rcnn = fit_model(rcnn, train_loader, val_loader, epochs=5)
# print(f"RCNN Test:   Acc={acc_rcnn:.4f}  F1={f1_rcnn:.4f}")


In [ ]:

# #@title 🧠 DRCNN (Dilated Residual CNN; approx DRI-RCNN)
# class DilatedResBlock(nn.Module):
#     def __init__(self, channels, dilation):
#         super().__init__()
#         self.conv1 = nn.Conv1d(channels, channels, 3, padding=dilation, dilation=dilation)
#         self.bn1 = nn.BatchNorm1d(channels)
#         self.conv2 = nn.Conv1d(channels, channels, 3, padding=dilation, dilation=dilation)
#         self.bn2 = nn.BatchNorm1d(channels)
#     def forward(self, x):
#         identity = x
#         out = F.relu(self.bn1(self.conv1(x)))
#         out = self.bn2(self.conv2(out))
#         return F.relu(out + identity)

# class DRCNN(nn.Module):
#     def __init__(self, vocab_size, embed_dim=128, base_channels=128, dilations=(1,2,4), num_classes=2, dropout=0.5):
#         super().__init__()
#         self.emb = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
#         self.proj = nn.Conv1d(embed_dim, base_channels, kernel_size=1)
#         self.blocks = nn.ModuleList([DilatedResBlock(base_channels, d) for d in dilations])
#         self.drop = nn.Dropout(dropout)
#         self.fc = nn.Linear(base_channels, num_classes)
#     def forward(self, ids):
#         x = self.emb(ids).transpose(1,2)
#         x = self.proj(x)
#         for blk in self.blocks:
#             x = blk(x)
#         x = F.adaptive_max_pool1d(x, 1).squeeze(-1)
#         x = self.drop(x)
#         return self.fc(x)

# drcnn = DRCNN(VOCAB_SIZE, EMBED_DIM)
# acc_drcnn, f1_drcnn = fit_model(drcnn, train_loader, val_loader, epochs=5)
# print(f"DRCNN Test:  Acc={acc_drcnn:.4f}  F1={f1_drcnn:.4f}")


In [ ]:

# #@title 🧠 MFNN (CNN + TF-IDF projection)
# TFIDF_MAX = 20000
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD

# tfidf = TfidfVectorizer(stop_words="english", max_features=TFIDF_MAX, ngram_range=(1,2))
# Xtf_train = tfidf.fit_transform(X_train)
# Xtf_val   = tfidf.transform(X_val)
# Xtf_test  = tfidf.transform(X_test)

# svd_dim = 200
# svd = TruncatedSVD(n_components=svd_dim, random_state=SEED)
# Z_train = svd.fit_transform(Xtf_train)
# Z_val   = svd.transform(Xtf_val)
# Z_test  = svd.transform(Xtf_test)

# class CNNPlusTFIDF(nn.Module):
#     def __init__(self, vocab_size, embed_dim=128, kernels=(3,4,5), channels=128, aux_dim=200, num_classes=2, dropout=0.5):
#         super().__init__()
#         self.emb = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
#         self.convs = nn.ModuleList([nn.Conv1d(embed_dim, channels, k) for k in kernels])
#         self.aux = nn.Linear(aux_dim, 128)
#         self.drop = nn.Dropout(dropout)
#         self.fc = nn.Linear(channels*len(kernels)+128, num_classes)
#     def forward(self, ids, aux):
#         x = self.emb(ids).transpose(1,2)
#         outs = [F.relu(c(x)).max(dim=2)[0] for c in self.convs]
#         x = torch.cat(outs, dim=1)
#         a = torch.relu(self.aux(aux))
#         x = torch.cat([x,a], dim=1)
#         x = self.drop(x)
#         return self.fc(x)

# class AuxDataset(Dataset):
#     def __init__(self, ids, masks, labels, aux_np):
#         self.ids=ids; self.masks=masks; self.labels=labels; self.aux=torch.tensor(aux_np, dtype=torch.float32)
#     def __len__(self): return len(self.labels)
#     def __getitem__(self, idx):
#         return self.ids[idx], self.masks[idx], self.labels[idx], self.aux[idx]

# train_aux_loader = DataLoader(AuxDataset(train_ids, train_mask, ytr, Z_train), batch_size=32, shuffle=True)
# val_aux_loader   = DataLoader(AuxDataset(val_ids, val_mask, yva, Z_val), batch_size=32)
# test_aux_loader  = DataLoader(AuxDataset(test_ids, test_mask, yte, Z_test), batch_size=32)

# def fit_model_aux(model, train_loader, val_loader, test_loader, epochs=5, lr=1e-3, wd=1e-4):
#     model = model.to(device)
#     opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
#     crit = nn.CrossEntropyLoss()
#     for ep in range(1, epochs+1):
#         model.train(); tr_loss=0.0
#         for ids, mask, labels, aux in train_loader:
#             ids=ids.to(device); labels=labels.to(device); aux=aux.to(device)
#             opt.zero_grad()
#             loss = crit(model(ids, aux), labels)
#             loss.backward(); opt.step()
#             tr_loss += loss.item()*ids.size(0)
#         va_acc, va_f1 = eval_aux(model, val_loader)
#         print(f"Epoch {ep}: train_loss={tr_loss/len(train_loader.dataset):.4f}  val_acc={va_acc:.4f}  val_f1={va_f1:.4f}")
#     te_acc, te_f1 = eval_aux(model, test_loader)
#     return te_acc, te_f1

# @torch.no_grad()
# def eval_aux(model, loader):
#     model.eval(); preds=[]; trues=[]
#     for ids, mask, labels, aux in loader:
#         ids=ids.to(device); aux=aux.to(device)
#         p = F.softmax(model(ids, aux), dim=-1)[:,1].cpu().numpy()
#         preds.extend(list((p>=0.5).astype(int))); trues.extend(list(labels.numpy()))
#     return accuracy_score(trues, preds), f1_score(trues, preds)

# mfnn = CNNPlusTFIDF(VOCAB_SIZE, EMBED_DIM, aux_dim=svd_dim)
# acc_mfnn, f1_mfnn = fit_model_aux(mfnn, train_aux_loader, val_aux_loader, test_aux_loader, epochs=5)
# print(f"MFNN Test:    Acc={acc_mfnn:.4f}  F1={f1_mfnn:.4f}")


In [ ]:

# #@title 🧠 SOM-CNN (tiny SOM on SVD-TFIDF)
# import numpy as np

# som_dim = (8,8)
# codebook_dim = Z_train.shape[1]
# rng = np.random.default_rng(SEED)
# codebook = rng.normal(0,0.1,size=(som_dim[0]*som_dim[1], codebook_dim))

# def bmu(x, codebook):
#     d = ((codebook - x)**2).sum(axis=1)
#     return int(np.argmin(d))

# def som_train(data, codebook, iters=400, init_lr=0.3, final_lr=0.05):
#     m = codebook.shape[0]
#     grid_xy = np.stack(np.unravel_index(np.arange(m), som_dim), axis=1)
#     max_radius = max(som_dim)
#     for t in range(iters):
#         lr = init_lr + (final_lr - init_lr) * (t/iters)
#         i = rng.integers(0, data.shape[0])
#         x = data[i]
#         j = bmu(x, codebook)
#         bj = grid_xy[j]
#         sigma = max(1.0, max_radius * (1 - t/iters))
#         d2 = ((grid_xy - bj)**2).sum(axis=1)
#         h = np.exp(-d2/(2*(sigma**2)))
#         codebook += lr * h[:,None] * (x - codebook)
#     return codebook

# print("Training small SOM...")
# codebook = som_train(Z_train, codebook)

# def som_feats(Z, codebook):
#     feats = np.zeros((Z.shape[0], codebook.shape[0]), dtype=np.float32)
#     for i in range(Z.shape[0]):
#         feats[i, bmu(Z[i], codebook)] = 1.0
#     return feats

# S_train = som_feats(Z_train, codebook)
# S_val   = som_feats(Z_val, codebook)
# S_test  = som_feats(Z_test, codebook)

# class SOMCNN(nn.Module):
#     def __init__(self, vocab_size, embed_dim=128, kernels=(3,4,5), channels=128, som_dim=64, num_classes=2, dropout=0.5):
#         super().__init__()
#         self.emb = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
#         self.convs = nn.ModuleList([nn.Conv1d(embed_dim, channels, k) for k in kernels])
#         self.sproj = nn.Linear(som_dim, 128)
#         self.drop = nn.Dropout(dropout)
#         self.fc = nn.Linear(channels*len(kernels)+128, num_classes)
#     def forward(self, ids, som_vec):
#         x = self.emb(ids).transpose(1,2)
#         outs = [F.relu(c(x)).max(dim=2)[0] for c in self.convs]
#         x = torch.cat(outs, dim=1)
#         s = torch.relu(self.sproj(som_vec))
#         x = torch.cat([x,s], dim=1)
#         x = self.drop(x)
#         return self.fc(x)

# class SomDataset(Dataset):
#     def __init__(self, ids, masks, labels, som_np):
#         self.ids=ids; self.masks=masks; self.labels=labels; self.som=torch.tensor(som_np, dtype=torch.float32)
#     def __len__(self): return len(self.labels)
#     def __getitem__(self, idx):
#         return self.ids[idx], self.masks[idx], self.labels[idx], self.som[idx]

# train_som_loader = DataLoader(SomDataset(train_ids, train_mask, ytr, S_train), batch_size=32, shuffle=True)
# val_som_loader   = DataLoader(SomDataset(val_ids, val_mask, yva, S_val), batch_size=32)
# test_som_loader  = DataLoader(SomDataset(test_ids, test_mask, yte, S_test), batch_size=32)

# def fit_model_som(model, train_loader, val_loader, test_loader, epochs=5, lr=1e-3, wd=1e-4):
#     model = model.to(device)
#     opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
#     crit = nn.CrossEntropyLoss()
#     for ep in range(1, epochs+1):
#         model.train(); tr_loss=0.0
#         for ids, mask, labels, som_vec in train_loader:
#             ids=ids.to(device); labels=labels.to(device); som_vec=som_vec.to(device)
#             opt.zero_grad()
#             loss = crit(model(ids, som_vec), labels)
#             loss.backward(); opt.step()
#             tr_loss += loss.item()*ids.size(0)
#         va_acc, va_f1 = eval_som(model, val_loader)
#         print(f"Epoch {ep}: train_loss={tr_loss/len(train_loader.dataset):.4f}  val_acc={va_acc:.4f}  val_f1={va_f1:.4f}")
#     te_acc, te_f1 = eval_som(model, test_loader)
#     return te_acc, te_f1

# @torch.no_grad()
# def eval_som(model, loader):
#     model.eval(); preds=[]; trues=[]
#     for ids, mask, labels, som_vec in loader:
#         ids=ids.to(device); som_vec=som_vec.to(device)
#         p = F.softmax(model(ids, som_vec), dim=-1)[:,1].cpu().numpy()
#         preds.extend(list((p>=0.5).astype(int))); trues.extend(list(labels.numpy()))
#     return accuracy_score(trues, preds), f1_score(trues, preds)

# somcnn = SOMCNN(VOCAB_SIZE, EMBED_DIM, som_dim=S_train.shape[1])
# acc_somcnn, f1_somcnn = fit_model_som(somcnn, train_som_loader, val_som_loader, test_som_loader, epochs=5)
# print(f"SOM-CNN Test: Acc={acc_somcnn:.4f}  F1={f1_somcnn:.4f}")


In [ ]:
# # ---- First cell in a fresh runtime for GPU path ----
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"  # reduce noise
# # (Do NOT set CUDA_VISIBLE_DEVICES here; we want GPU visible.)


In [ ]:
# # ---- Run this as the FIRST cell after Restart runtime ----
# # Force CPU for the whole process (must be BEFORE importing torch)
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # no GPUs visible
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# import numpy as np
# import torch
# from torch import nn
# from torch.utils.data import Dataset, DataLoader
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from sklearn.metrics import accuracy_score, f1_score

# # Assumes X_train, X_val, X_test, y_train, y_val, y_test are already created later.
# # If not, put your CSV-loading + stratified split cells *above* this block or rerun them now.

# class BertTextDataset(Dataset):
#     def __init__(self, texts, labels, tok, max_len=256):
#         self.texts = [str(t) for t in texts]
#         self.labels = labels
#         self.tok = tok
#         self.max_len = max_len
#     def __len__(self): return len(self.texts)
#     def __getitem__(self, i):
#         enc = self.tok(
#             self.texts[i],
#             padding="max_length",
#             truncation=True,
#             max_length=self.max_len,
#             return_tensors="pt",
#         )
#         item = {k: v.squeeze(0) for k, v in enc.items()}
#         item["labels"] = torch.tensor(int(self.labels[i]), dtype=torch.long)
#         return item

# def run_bert_cpu(X_train, y_train, X_val, y_val, X_test, y_test, epochs=3, bs=16, lr=2e-5):
#     device = torch.device("cpu")
#     tok = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
#     ds_tr = BertTextDataset(X_train, y_train, tok)
#     ds_va = BertTextDataset(X_val,   y_val,   tok)
#     ds_te = BertTextDataset(X_test,  y_test,  tok)

#     tr = DataLoader(ds_tr, batch_size=bs, shuffle=True)
#     va = DataLoader(ds_va, batch_size=bs)
#     te = DataLoader(ds_te, batch_size=bs)

#     model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
#     model.to(device)
#     opt = torch.optim.AdamW(model.parameters(), lr=lr)
#     loss_fn = nn.CrossEntropyLoss()

#     def evaluate(loader):
#         model.eval()
#         probs, gold = [], []
#         with torch.no_grad():
#             for batch in loader:
#                 for k in ["input_ids", "attention_mask", "labels"]:
#                     batch[k] = batch[k].to(device)
#                 out = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
#                 p = torch.softmax(out.logits, dim=-1)[:,1].cpu().numpy()
#                 probs.extend(p.tolist())
#                 gold.extend(batch["labels"].cpu().numpy().tolist())
#         preds = (np.array(probs) >= 0.5).astype(int)
#         return accuracy_score(gold, preds), f1_score(gold, preds)

#     # Train loop (CPU)
#     for ep in range(1, epochs+1):
#         model.train()
#         running = 0.0
#         for batch in tr:
#             for k in ["input_ids", "attention_mask", "labels"]:
#                 batch[k] = batch[k].to(device)
#             out = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["labels"])
#             loss = out.loss
#             opt.zero_grad(); loss.backward(); opt.step()
#             running += loss.item() * batch["input_ids"].size(0)
#         va_acc, va_f1 = evaluate(va)
#         print(f"Epoch {ep}: train_loss={running/len(ds_tr):.4f}  val_acc={va_acc:.4f}  val_f1={va_f1:.4f}")

#     te_acc, te_f1 = evaluate(te)
#     print(f"BERT Base — Test: Accuracy={te_acc:.4f}  F1={te_f1:.4f}")
#     return te_acc, te_f1

# # Example usage *after* you’ve created X_train/X_val/X_test/y_*:
# # acc_bert, f1_bert = run_bert_cpu(X_train, y_train, X_val, y_val, X_test, y_test)
# # results["BERT_Base"] = (acc_bert, f1_bert)   # if you keep a comparison dict


In [ ]:
# # Example usage *after* you’ve created X_train/X_val/X_test/y_*:
# acc_bert, f1_bert = run_bert_cpu(X_train, y_train, X_val, y_val, X_test, y_test)
# # results["BERT_Base"] = (acc_bert, f1_bert)   # if you keep a comparison dict

In [ ]:
# results["BERT_Base"] = (acc_bert, f1_bert)

In [ ]:
print(results)

plot like the paper

In [ ]:


# def add_result(name, acc, f1):
#     """
#     name: model name as a string (how you want it to appear on the plot)
#     acc, f1: either fractions (0–1) or percentages (70–100)
#     in_percent: set True if you're passing 90.31 not 0.9031
#     """
#     results[name] = (float(acc), float(f1))

# # Example: add YOUR measured numbers (replace with your actual values)


# add_result("RCNN",                            0.8000, 0.8037)
# # add_result("GRNN-CNN",                        84.15, 84.17)
# add_result("DRI-RCNN",                        0.703,0.6025)
# add_result("LDA with TextCNN",                0.8063, 0.8239)
# add_result("MFNN",                            0.8125, 0.7959)
# add_result("SOM-CNN",                         0.8187, 0.8284)

# # If you already computed RoBERTa+LSTM earlier:
# # results["RoBERTa_LSTM"] = (acc_rb, f1_rb)
# # or set it manually:
# print(results)


In [ ]:
# import pandas as pd

# # Fix a canonical order for plotting/table
# models_order = [
#     "SVM_bigram",
#     "SVM_unigram",
#     "SAGA",
#     "RCNN",
#     "GRNN-CNN",
#     "DRI-RCNN",
#     "BERT_Base",
#     "LDA with TextCNN",
#     "MFNN",
#     "SOM-CNN",
#     "RoBERTa_LSTM",
# ]

# rows = []
# for m in models_order:
#     if m in results:
#         acc, f1 = results[m]
#         rows.append({"Model": m, "Accuracy (%)": round(acc*100, 2), "F1-score (%)": round(f1*100, 2)})

# df_compare = pd.DataFrame(rows)
# display(df_compare)
# df_compare.to_csv("all_model_comparison.csv", index=False)
# print("Saved: all_model_comparison.csv")


In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# labels, accs, f1s = [], [], []
# for m in models_order:
#     if m in results:
#         a, f = results[m]
#         labels.append(m)
#         accs.append(a*100.0)
#         f1s.append(f*100.0)

# x = np.arange(len(labels))
# width = 0.35

# plt.figure(figsize=(12,5))
# plt.bar(x - width/2, accs, width, label='Op-Spam dataset Accuracy')
# plt.bar(x + width/2, f1s,  width, label='Op-Spam dataset F1-score')

# plt.xticks(x, labels, rotation=35, ha='right')
# plt.ylabel('Score (%)')
# plt.ylim(70, 100)          # match the paper’s scale if you like
# plt.title('Performance Comparison Under the Op-Spam dataset')
# plt.legend()
# plt.tight_layout()
# plt.savefig("opspam_comparison_bars.png", dpi=180)
# plt.show()

# print("Saved: opspam_comparison_bars.png")
